In [1]:
import mlflow
import os
import pickle

In [2]:

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("nyc-taxi-regression-assignment")

<Experiment: artifact_location='/Users/hsuyab/Documents/programming/work_related/mlops_zoomcap/week2_new/artifact/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-regression-assignment', tags={}>

## Question1: mlflow version

In [3]:
print('mlflow verion:', mlflow.__version__)

mlflow verion: 1.26.1


## Question2: Download and preprocess the data

#### Preprocessing the data

In [4]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output


In [5]:
!ls -l ./output

total 18328
-rw-r--r--  1 hsuyab  staff   305256 Jun  2 21:56 dv.pkl
-rw-r--r--  1 hsuyab  staff  2805197 Jun  2 21:56 test.pkl
-rw-r--r--  1 hsuyab  staff  2661149 Jun  2 21:56 train.pkl
-rw-r--r--  1 hsuyab  staff  2166527 Jun  2 21:56 valid.pkl


In [6]:
print('Number of files saved in output folder:', len(os.listdir('./output')))

Number of files saved in output folder: 4


### Training a model with autolog

In [7]:
!python train.py

2022/06/02 21:57:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/hsuyab/miniconda3/envs/mlops/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


### Run hpo

In [25]:
!python hpo.py

2022/06/02 22:39:34 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|█████████| 50/50 [04:44<00:00,  5.70s/trial, best loss: 6.6284257482044735]


In [26]:
from mlflow.tracking import MlflowClient
client = MlflowClient(tracking_uri='http://127.0.0.1:5000')

In [34]:
runs = client.search_runs(experiment_ids=1, max_results=5, order_by=['metrics.rmse ASC'])

In [35]:
for run in runs:
    print(f"run id: {run.info.run_uuid}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 87c4560f6d944f0a83db6f2d40e55790, rmse: 6.6284
run id: 1abc983c57f946f8985d45da3bd9947e, rmse: 6.6297
run id: 33dcf2c75d3f4aac9aff2cd50e2e9ccd, rmse: 6.6299
run id: 65c392e5959e4f829630d6b5f2230511, rmse: 6.6299
run id: 300deb7ea310459282d56a9abe100fc8, rmse: 6.6299


### Run the final register_model.py

In [29]:
!python register_model.py

2022/06/02 22:44:36 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2022/06/02 22:44:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/hsuyab/miniconda3/envs/mlops/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Traceback (most recent call last):
  File "register_model.py", line 98, in <module>
    run(args.data_path, args.top_n)
  File "register_model.py", line 76, in run
    mlflow.register_model(
TypeError: register_model() got an unexpected keyword argument 'registered_model_name'
